In [1]:
!pip install pyspark

     ------------------------------------ 316.9/316.9 MB 718.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 200.5/200.5 kB 468.6 kB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425367 sha256=1650c4a0f300b71f15b4e029ecb22d2604d45dfd5928b8eb6e11ea4030405118
  Stored in directory: c:\users\naima\appdata\local\pip\cache\wheels\23\6e\46\d22b2a968d8ca06247fd42f04d40a95741472068160bdd46dd
Successfully built pyspark


In [2]:
!pip install pyspark[pandas_on_spark] plotly 

     --------------------------------------- 15.6/15.6 MB 74.9 kB/s eta 0:00:00
     -------------------------------------- 24.4/24.4 MB 139.0 kB/s eta 0:00:00


In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark= SparkSession.builder.appName('Customers').getOrCreate()

In [3]:
dataset=spark.read.csv("customer_churn.csv",inferSchema=True,header=True)

In [4]:
dataset.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
dataset.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
from pyspark.ml.feature import VectorAssembler 

In [7]:
assembler=VectorAssembler(inputCols=['Age', 
 'Total_Purchase', 
 'Account_Manager', 
 'Years', 
 'Num_Sites'], 
outputCol="features")

In [8]:
output=assembler.transform(dataset) 

In [9]:
df_final = output.select("features","churn") 
df_final.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [10]:
train,test=df_final.randomSplit([0.7,0.3], seed=42)

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
lr = LogisticRegression(labelCol="churn") 

In [13]:
lrm = lr.fit(train) 
lrm_summary = lrm.summary 

In [14]:
lrm_summary.predictions.show() 

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.55979963242690...|[0.98964420919358...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[4.67536757133125...|[0.99076400091638...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.28230440567531...|[0.99813439848473...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[5.32554251161008...|[0.99515784990742...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[7.59026200904824...|[0.99949490661754...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.09749022712613...|[0.98365720318515...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.06733742325029...|[0.98316533957374...|       0.0|
|[29.0,8688.17,1.0...|  1.0|[2.71962101743711...|[0.93817455523842...|       0.0|
|[29.0,9378.24,0.0...|  0.0|[4.73007562142385...|[0.99125140974466...|       0.0|
|[29.0,12711.15,

In [15]:
lrm_summary.predictions.describe().show() 

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               667|                667|
|   mean|0.1634182908545727|0.12293853073463268|
| stddev|0.3700243606477147|0.32861306618408714|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [17]:
pred_labels = lrm.evaluate(test)

In [18]:
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.79106248132020...|[0.68805942038441...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.61026688857555...|[0.83344843709585...|       0.0|
|[28.0,11204.23,0....|    0|[1.97148345642762...|[0.87777036176974...|       0.0|
|[28.0,11245.38,0....|    0|[3.75331012854098...|[0.97709682330471...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.42202807822147...|[0.98813267415840...|       0.0|
|[29.0,10203.18,1....|    0|[3.71080419195967...|[0.97612605863733...|       0.0|
|[29.0,11274.46,1....|    0|[4.39058463869248...|[0.98775823667280...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.55749267043437...|[0.97228008128416...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.76304568829245...|[0.99686830940139...|       0.0|
|[30.0,8874.83,0

In [19]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn') 
auc = eval.evaluate(pred_labels.predictions) 

In [20]:
auc 

0.7456808943089431

In [22]:
customers = spark.read.csv('new_customers_test.csv',inferSchema=True, header=True) 

In [23]:
final_model = lr.fit(df_final) 
customers.printSchema() 

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [24]:
customers_valid = assembler.transform(customers) 
customers_valid.printSchema() 
res = final_model.transform(customers_valid)

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
res.select('Company','prediction').show() 
res.select('Names','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

+--------------+----------+
|         Names|prediction|
+--------------+----------+
| Andrew Mccall|       0.0|
|Michele Wright|       1.0|
|  Jeremy Chang|       1.0|
|Megan Ferguson|       1.0|
|  Taylor Young|       0.0|
| Jessica Drake|       1.0|
+--------------+----------+

